# Inference

## Import modules

In [1]:
import os
from glob import glob

import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import Model
from tensorflow.keras import regularizers

## Declare paths and globals

In [2]:
WEIGHTS_PATH = 'final_weights.h5'
TEST_DATA_PATH = "UrbanSound8k/tfRecords/fold"
TEST_BATCH_SIZE = 64

## Load tfRecord utils

In [3]:
AUTO = tf.data.experimental.AUTOTUNE

def parse_tfrecord(example):
    ''' It is strange you need to use tf.string to read in an image '''
    feature_description = {
        "image": tf.io.FixedLenFeature([], tf.string),
        "target": tf.io.FixedLenFeature([], tf.int64),
    }
    example = tf.io.parse_single_example(example, feature_description)
    example["image"] = tf.io.decode_png(example["image"], channels=3)
    image = example['image']
    target = example['target']
    target_hot = tf.one_hot(target, 10)
    return (image,target_hot)

def get_dataset(record_files):
    dataset = tf.data.TFRecordDataset(record_files, buffer_size=100)
    dataset = dataset.map(parse_tfrecord, num_parallel_calls=AUTO)
    dataset = dataset.prefetch(AUTO)
    return dataset

## Load model and weights 

In [4]:
def getEfficientNetB4(input_shape=(128, 250, 3)):
    # Load base model
    base_model = tf.keras.applications.EfficientNetB4(
        weights="imagenet",  # Load weights pre-trained on ImageNet.
        input_shape=input_shape,
        include_top=False,
    )
    feat_ex = tf.keras.Model(base_model.input, base_model.output)

    # Add new layers
    inputs = tf.keras.Input(input_shape)
    x = feat_ex(inputs)
    x = layers.GlobalAveragePooling2D()(x)
    x = layers.Dropout(0.5)(x)
    yh = layers.Dense(
        10, kernel_regularizer=regularizers.l2(0.0001), activation="softmax"
    )(x)
    model = tf.keras.Model(inputs, yh)
    print(model.summary())
    return model

In [5]:
model = getEfficientNetB4()
model.load_weights(WEIGHTS_PATH)

opt = tf.keras.optimizers.Adam(learning_rate=1e-4)
model.compile(optimizer=opt,loss=tf.keras.losses.CategoricalCrossentropy(),metrics=["accuracy"],)

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 128, 250, 3)]     0         
_________________________________________________________________
model (Functional)           (None, 4, 8, 1792)        17673823  
_________________________________________________________________
global_average_pooling2d (Gl (None, 1792)              0         
_________________________________________________________________
dropout (Dropout)            (None, 1792)              0         
_________________________________________________________________
dense (Dense)                (None, 10)                17930     
Total params: 17,691,753
Trainable params: 17,566,546
Non-trainable params: 125,207
_________________________________________________________________
None


## Test loaded weights over UrbanSound8k folds

In [6]:
folds = []
for f in range(1, 11):
    fold = [i for i in range(1, 11) if i != f]
    folds.append((f, fold))
print(folds)

[(1, [2, 3, 4, 5, 6, 7, 8, 9, 10]), (2, [1, 3, 4, 5, 6, 7, 8, 9, 10]), (3, [1, 2, 4, 5, 6, 7, 8, 9, 10]), (4, [1, 2, 3, 5, 6, 7, 8, 9, 10]), (5, [1, 2, 3, 4, 6, 7, 8, 9, 10]), (6, [1, 2, 3, 4, 5, 7, 8, 9, 10]), (7, [1, 2, 3, 4, 5, 6, 8, 9, 10]), (8, [1, 2, 3, 4, 5, 6, 7, 9, 10]), (9, [1, 2, 3, 4, 5, 6, 7, 8, 10]), (10, [1, 2, 3, 4, 5, 6, 7, 8, 9])]


In [7]:
for i, fold in enumerate(folds):
    print("Fold {}".format(i + 1))
    fold_index = i
    test_path = TEST_DATA_PATH + str(fold[0]) + ".tfrec"
    test_dataset = get_dataset(test_path).batch(TEST_BATCH_SIZE)
    print("Evaluate on test data")
    results = model.evaluate(test_dataset)
    print(results)

Fold 1
Evaluate on test data
14/14 [==============================] - 45s 3s/step - loss: 0.0974 - accuracy: 0.9580
[0.12225078046321869, 0.9507445693016052]
Fold 2
Evaluate on test data
14/14 [==============================] - 41s 3s/step - loss: 0.0634 - accuracy: 0.9797
[0.06343276798725128, 0.9797297120094299]
Fold 3
Evaluate on test data
15/15 [==============================] - 42s 3s/step - loss: 0.0151 - accuracy: 0.9957
[0.015061570331454277, 0.9956756830215454]
Fold 4
Evaluate on test data
16/16 [==============================] - 45s 3s/step - loss: 0.0096 - accuracy: 0.9980
[0.009582444094121456, 0.9979798197746277]
Fold 5
Evaluate on test data
15/15 [==============================] - 41s 3s/step - loss: 0.0064 - accuracy: 1.0000
[0.0063959634862840176, 1.0]
Fold 6
Evaluate on test data
13/13 [==============================] - 36s 3s/step - loss: 0.0073 - accuracy: 0.9988
[0.007315364666283131, 0.9987849593162537]
Fold 7
Evaluate on test data
14/14 [==========================

## Make into tf lite model 
https://www.tensorflow.org/lite